<a href="https://colab.research.google.com/github/khaoulaloudiyi/QuickNews/blob/master/PresseAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dateparser

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive//MediaInsights

/content/drive/My Drive/MediaInsights


In [ ]:
!pwd

/content/drive/My Drive/MediaInsights


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from datetime import datetime
import pandas as pd
import csv
import dateparser
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier
from collections import Counter 
from string import punctuation
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stop_words
import spacy 
from spacy.lang.en.stop_words import STOP_WORDS
import nltk 
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
swa = stopwords.words('arabic')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Cette partie doit être executer une seule fois pour créer le header des deux fichiers csv

#Ce fichier joue un rôle principal pour la mise à jour puisqu'il contient le champ lastDate qui réfère à la date ou la dernière mise à jour a été effectué et le champ maxid pour faciliter la recherche du plus grand Id 
with open('presse_lastUpdate.csv','w',encoding="utf-8") as csv_file:
    fieldnames=['country', 'Publication', 'maxId', 'lastDate']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
#Ce fichier est réservé pour stocker les différents articles récupéré grâce au scraping 
with open('ArticlesAR.csv','w',encoding="utf-8") as csv_file:
    fieldnames=['Id','Title','Author','Date','Publication','URL','Content','Language','Country', "sentiment", "Summary"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()


In [ ]:
#D'abord il faut comprendre la structure des sites web de la presse, chaque presse possède un archive où tous les articles sont stocké, sous forme d'une dizaine d'articles dans plusieurs pages, 
#chaque page de ce dernier contient plusieurs articles référencé par des liens, et tous ces articles possède le même format et contient l'ensemble des informations sauf 
#l'information sur le "sentiment"

#Le principe du Scraping qu'on a utilisé est le suivant "exemple avec Hespress Ar mais la même chose se répète avec les autres presses":
#d'abord on effectue une étude pour comprendre la structure d'une page de l'archive (code html) afin de récupérer les liens des articles seulement
#apres on analyse la structure d'un de ces articles (html) , On parcour l'ensemble des pages de l'archive grâce à la fonction "add_newArticles_hespres" cette
#fonction utilise la variable p qui représente le numéro de page de l'archive a scrapé, on scrape cette page puis on collecte les liens
#href des articles et pour chaque lien on scrape l'article avec la fonction "scrap_article_Hespress" et on l'ajout à la dataset Articles


In [ ]:
#tokenizing sentences into words
def tokenizer(s):
    tokens = []
    for word in s.split(' '):
        tokens.append(word.strip().lower()) #strip remove spaces at the beginning and at the end of the string
    return tokens

#tokenizing document into sentences 
def sent_tokenizer(s):
    sents = []
    for sent in s.split('.'):
        sents.append(sent.strip())
    return sents

def count_words(tokens):
    word_counts = {}
    for token in tokens:
        if token not in stop_words and token not in STOP_WORDS and token not in swa and token not in punctuation:
            if token not in word_counts.keys():
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

def word_freq_distribution(word_counts):
    freq_dist = {}
    max_freq = max(word_counts.values())
    for word in word_counts.keys():  
        freq_dist[word] = (word_counts[word]/max_freq)
    return freq_dist

def score_sentences(sents, freq_dist, max_len=40):
    sent_scores = {}  
    for sent in sents:
        words = sent.split(' ')
        for word in words:
            if word.lower() in freq_dist.keys():
                if len(words) < max_len:
                    if sent not in sent_scores.keys():
                        sent_scores[sent] = freq_dist[word.lower()]
                    else:
                        sent_scores[sent] += freq_dist[word.lower()]
    return sent_scores

def summarize(sent_scores, k):
    top_sents = Counter(sent_scores) 
    summary = ''
    scores = []
    
    top = top_sents.most_common(k)
    for t in top: 
        summary += t[0].strip()+'. '
        scores.append((t[1], t[0]))
    return summary[:-1], scores

In [ ]:
#hespress
def scrap_article_Hespress(site, date_max, Id, writer):
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(site,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page)
    #Pour les pages ou il n y a pas de titre on ne la traîte pas
    if not soup.select('h1.page_title'):
        return True, date_max, Id
    #pour Obtenir le titre
    title = soup.select('h1.page_title')[0].text.strip()
    
    #Le contenu est stocké dans la div ou l id = "article_holder" dans plusieurs paragraphes (balises p) 
    content_a = soup.findAll("div", {"id": "article_holder"})
    content = ''
    for x in content_a:
        for c in x.findAll('p'):
            #on stocke la totalite du contenu dans la variable contenu
            content += c.text + "\n"

    scrap_author=soup.findAll("span",{"class":"story_author"})
    longueur=len(scrap_author)
    txt=scrap_author[longueur-1].text
    var_author = txt.split('-')
    #cette partie adapté a la structure des auteurs de hespress
    if len(var_author)==1:
        author=var_author[0]
    else:
        author=var_author[1]    
    
    #la date de l article
    date_article=soup.find("span",{"class":"story_date"}).text
    #on regle les problemes de la langue arabe par respecter les normes
    date_pub = date_article.replace("يوليوز","يوليو")
    date_pub = date_pub.replace("غشت","اغسطس")
    date_pub = date_pub.replace("نونبر","نوفمبر")
    date_pub = date_pub.replace("شتنبر","سبتمبر")
    date_pub = date_pub.replace("دجنبر","ديسمبر")
    date_pub = date_pub.replace("ماي","مايو")

    
    date_publication =  dateparser.parse(date_pub)
    language='Ar'
    country='Morocco'
    publication='Hespress'
    #si la date est inferieur a date_max qui correspand a la date ou la dernier mis a jour est effectué on arrête le scraping 
    if(date_publication <= date_max):
        return False, date_publication, Id 
    Id = Id +1
    
    #analyse de sentiment "notre propre proposition"
    en_blob = TextBlob(str(content))
    sent = en_blob.sentiment.polarity
    if sent <-0.05 :
        sentiment = "Negative"
    elif sent > -0.05 and sent < 0.05 :
        sentiment = "Neutre"
    else :
        sentiment = "Positive"
    #Summarinzing
    tokens = tokenizer(content)
    sents = sent_tokenizer(content)
    word_counts = count_words(tokens)
    freq_dist = word_freq_distribution(word_counts)
    sent_scores = score_sentences(sents, freq_dist)
    summary, summary_sent_scores = summarize(sent_scores, 3)

    #on l'ecrit dans le fichier csv 
    writer.writerow({'Id': Id,'Title': title,'Author': author,'Date':date_publication,'Publication': publication,'URL': site,'Content': content,'Language': language,'Country': country, 'sentiment':sentiment, 'Summary':summary})
    return True, date_publication, Id

def add_newArticles_hespress(Id, date_max): 
    #variable pour parcourir les pages de l'archive
    p =1
    stay = True
    fieldnames=['Id','Title','Author','Date', 'Publication','URL','Content','Language','Country','sentiment','Summary']
    #Ovrir le fichier Articles une seule fois
    with open('ArticlesAR.csv','a+',encoding="utf-8") as csv_file:
        article_writer = csv.writer(csv_file)
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        lastDate= date_max
        while stay: 
            site= "https://www.hespress.com/24-heures/index."+str(p)+".html"
            p = p+1
            hdr = {'User-Agent': 'Mozilla/5.0'}
            req = Request(site,headers=hdr)
            page = urlopen(req)
            soup = BeautifulSoup(page)
            
            #Chercher tous les liens 
            links = soup.findAll("a")
            i=0
            for link in links:
                if "#box_most_popular" == link["href"] :
                    break
                if 'index.1.html' not in link["href"] and 'https' not in link["href"] and 'http' not in link["href"]: 
                    if i%3 == 0 :
                        #Effectuer le scrapping d'un artcile mais puisque il est repete 3 fois on le utilise la variable i pour 
                        #ne pas le duppliquer
                        stay, date, Id = scrap_article_Hespress('https://www.hespress.com'+link["href"], date_max, Id, writer)
                        if date > lastDate:
                            #Obtenir la date la plus recente
                            lastDate = date
                        if not stay :
                            break
                    i = i+1
            if i == 0 :
                break
    return Id,lastDate

def update_hespress():
    fieldnames=['country', 'Publication', 'maxId', 'lastDate']
    #id_max et lastdate represent les informations les plus importants pour la mise a jour 
    with open('presse_lastUpdate.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        readData = [row for row in reader]
        found = False
        #l'initialisation des 2 variables sert a faire marcher le code meme si c'est la premier fois
        id_max = 0 
        lastdate = datetime(1 ,1 ,1 , 1, 1)
        for row in readData:
            if id_max < int(row['maxId']):
                id_max = int(row['maxId'])
            if row['country'] == "Morocco" and row['Publication'] == "Hespress" :
                lastdate = pd.to_datetime(row['lastDate'])  
                found = True
        
        #Effectuer la collection des données
        id_max, lastdate = add_newArticles_hespress(id_max, lastdate)
    
    #stocké les nouveaux variables id_max et lastdate
    if found :
        with open('presse_lastUpdate.csv', "w", newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for row in readData:
                if row['country'] == "Morocco" and row['Publication'] == "Hespress" :
                    row['maxId'] = id_max
                    row['lastDate'] = lastdate
                    break
            writer.writerows(readData)
    if not found:
        with open('presse_lastUpdate.csv', "a+", newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow({'country' :'Morocco', 'Publication' : 'Hespress',  'maxId' : id_max, 'lastDate' :lastdate})


In [ ]:
def update():
    #update_hespressFR()
    #update_LeMatin()
    update_hespress()
    #updates_TheAtlantic()
    #updates_ELMundo()
update()

KeyboardInterrupt: ignored